# Setup and Package Installation
Install and load required R packages: ipumsr, tidyverse, srvyr, tidycensus, duckplyr, duckdb, DBI, hudr. Set up environment variables for HUD API access.

In [1]:
# Install required packages if not already installed
if (!require("ipumsr")) install.packages("ipumsr")
if (!require("tidyverse")) install.packages("tidyverse")
if (!require("srvyr")) install.packages("srvyr")
if (!require("tidycensus")) install.packages("tidycensus")
if (!require("duckplyr")) install.packages("duckplyr")
if (!require("duckdb")) install.packages("duckdb")
if (!require("DBI")) install.packages("DBI")
if (!require("hudr")) install.packages("hudr")

# Load required libraries
library(ipumsr)
library(tidyverse)
library(srvyr)
library(tidycensus)
library(duckplyr)
library(duckdb)
library(DBI)
library(hudr)

# Set up environment variables for HUD API access
hud_key <- Sys.getenv("HUD_API_KEY")

# Set up DuckDB connection

duckconn <- dbConnect(duckdb::duckdb(), "data/acs_00_23.duckdb")


Loading required package: ipumsr
Loading required package: tidyverse
── Attaching core tidyverse package
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ──────────────────────
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the ]8;;http://conflicted.r-lib.org/conflicted package]8;; to force all conflicts to become errors
Loading required package: srvyr

Attaching package: ‘srvyr’

The following object is masked from ‘package:stats’:

    filter

Loading required package: tidycensus
Loading required package: duckplyr
The duckplyr package is configured
to fall back to dplyr when it
encounters an incompatibility.
Fallback events can be collected
and uploaded for analysis to guide
future development. By default,
data will be collected but no data
will be uploaded.
ℹ Automatic fallback uploading is
  not co

# Data Collection Functions
Create functions to fetch Median Family Income data from ACS API (2005-2023) and process historical data (2000-2004) from FTP downloads.

In [2]:
# Function to fetch Median Family Income data from ACS API (2005-2023)
fetch_acs_mfi <- function(acs_years) {
  acs_mfi <- map(
    acs_years,
    ~ get_acs(
      geography = "state",
      variables = "B19113_001",
      state = "CO",
      year = .x,
      survey = "acs1"
    ) |>
      mutate(year = .x, .before = 1)
  ) |>
    bind_rows()

  return(acs_mfi)
}

# Function to process historical Median Family Income data (2000-2004) from FTP downloads
process_historical_mfi <- function(file_path) {
  mfi_00_04 <- read_csv(file_path) |>
    filter(Stub == "Median family income (dollars)" & `Geographic Name` == "Colorado") |>
    select(NAME = `Geographic Name`, contains("Estimate")) |>
    pivot_longer(
      cols = contains("Estimate"),
      names_to = "year",
      values_to = "estimate",
      values_transform = as.numeric
    ) |>
    mutate(
      GEOID = "08",
      variable = "B19113_001",
      year = as.integer(str_extract(year, "\\d{4}"))
    ) |>
    as_duckdb_tibble()

  return(mfi_00_04)
}


In [3]:
# Function to fetch Median Family Income data from ACS API (2005-2023)
if (file.exists("data/combined_mfi.parquet")) {
  combined_mfi <- tbl(duckconn, "combined_mfi")
} else {
# Define ACS years and file path for historical data
acs_years <- c(2005:2019, 2021:2023)
historical_file_path <- "MultiYearProfiles0402004.csv"

# Fetch and process Median Family Income data
acs_mfi <- fetch_acs_mfi(acs_years)
mfi_00_04 <- process_historical_mfi(historical_file_path)

# Combine ACS and historical data
combined_mfi <- bind_rows(
  acs_mfi, 
  mfi_00_04, 
  acs_mfi |> 
    filter(year %in% c(2019:2021)) |>  
    summarize(
      estimate = mean(estimate),
      year = 2020,
      .by = !c(year, estimate, moe))
) |> 
  arrange(year)

# Save combined data to file
compute_parquet(combined_mfi, "data/combined_mfi.parquet")
dbWriteTable(duckconn, "combined_mfi", collect(combined_mfi), overwrite = TRUE)
}



# Load and Process ACS Data
Load ACS PUMS data using ipumsr package, process the data extracts, and set up DuckDB connection for efficient data handling.

In [14]:
ipums_file <- "usa_00074.xml"
file_loc <- paste0(
  "data/ipums_raw/",
  ipums_file
)
ipums_ddi <- read_ipums_ddi(file_loc)

acs_var_info <- ipums_var_info(ipums_ddi)

In [ ]:

if (file.exists("data/acs_00_23.duckdb")) {
# Collect data from DuckDB
acs_00_23 <- tbl(duckconn, "acs_00_23")
} else {

# Load the IPUMS data from RDS file if it exists
if (file.exists("data/acs_00_23.parquet")) {
  acs_00_23 <- duckplyr::read_parquet_duckdb("data/acs_00_23.parquet") |> 
    collect()
} else {
  
  if (file.exists(file_loc)) {
    acs_00_23 <- ipums_ddi |>
      read_ipums_micro() |> 
        as_duckplyr_tibble()
  } else {
    duckplyr::methods_overwrite()
    acs_samples <- get_sample_info("usa") |>
      filter(str_detect(name, pattern = "us20\\d{2}a")) |>
      pull(name)
    acs_extract <- define_extract_micro(
      collection = "usa",
      description = "ACS 1 year samples in Colorado of income variables, all samples since 2000",
      samples = acs_samples,
      variables = list(
        var_spec("STATEFIP", case_selections = "08"),
        "COUNTYFIP",
        "PUMA",
        "NUMPREC",
        "CPI99",
        "OWNERSHP",
        "OWNCOST",
        "RENT",
        "RENTGRS",
        "RELATE",
        "RACE",
        "SEX",
        "HISPAN",
        "BPL",
        "CITIZEN",
        "HHTYPE",
        "HHINCOME",
        "INCTOT",
        "INCWAGE",
        "INCBUS00",
        "INCSS",
        "INCWELFR",
        "INCINVST",
        "INCSUPP",
        "INCOTHER",
        "INCEARN",
        "POVERTY",
        "CBPOVERTY",
        "AGE",
        "FAMUNIT",
        "FAMSIZE",
        "NCHILD",
        "NCHLT5",
        "FTOTINC",
        "BEDROOMS",
        "MOVEDIN",
        "REPWT"
      )
    ) |>
      submit_extract() |>
      wait_for_extract() |>
      download_extract() |>
      read_ipums_micro()
  }
  acs_00_23 <- zap_labels(acs_00_23)
  duckplyr::compute_parquet(acs_00_23, "data/acs_00_23.parquet")
  # Write acs_00_23 to DuckDB database
  dbWriteTable(duckconn, "acs_00_23", acs_00_23, overwrite = TRUE)
}
}



methods_overwrite()

# Load and Process IPUMS Data
Extract and process IPUMS microdata, including household characteristics, income levels, and housing costs.

# Calculate AMI Groups
Create AMI calculations including household size adjustments, percentage of median family income, and cost burden classifications.

In [5]:
# Calculate AMI Groups

# Modify IPUMS Data to create AMI calculations
acs_amis <- acs_00_23 |>
  left_join(
    combined_mfi |>
      select("YEAR" = "year", mfi = estimate)
  ) |>
  collect() |> 
  as_duckdb_tibble() |> 
  mutate(
    nadults = sum(AGE >= 18),
    nchildren = sum(AGE < 18),
    check = NUMPREC - nadults - nchildren,
    .by = c("YEAR", "SERIAL")
  ) |>
  mutate(
    hhsize = nadults + nchildren,
    household_type_id = factor(case_when(
      nadults == 1 & NCHILD == 0 ~ "1",
      nadults == 1 & NCHILD > 0 ~ "2",
      nadults > 1 & NCHILD == 0 ~ "3",
      nadults > 1 & NCHILD > 0 ~ "4"
    )),
    household_type_description = factor(case_when(
      nadults == 1 & NCHILD == 0 ~ "One adult with no children",
      nadults == 1 & NCHILD > 0 ~ "One adult with children",
      nadults > 1 & NCHILD == 0 ~ "More than one adult with no children",
      nadults > 1 & NCHILD > 0 ~ "More than one adult with children"
    )),
    age_group_id = cut(
      AGE,
      breaks = c(
        0,
        18,
        25,
        45,
        65,
        Inf
      ),
      labels = c(
        "5",
        "1",
        "2",
        "3",
        "4"
      ),
      right = FALSE,
      ordered_result = TRUE
    ),
    age_group_description = cut(
      AGE,
      breaks = c(
        0,
        18,
        25,
        45,
        65,
        Inf
      ),
      labels = c(
        "Under 18",
        "18-24",
        "25-44",
        "45-64",
        "65 and over"
      ),
      right = FALSE,
      ordered_result = TRUE
    ),
    hhadj = case_when(
      hhsize < 4 ~ 1 - (4 - hhsize) * .1,
      hhsize > .4 ~ 1 + (hhsize - 4) * .08,
      .default = 1
    ),
    mfi_hh = mfi * hhadj,
    pct_mfi = HHINCOME / mfi_hh,
    ami_group = cut(
      pct_mfi,
      breaks = c(
        0,
        .3,
        .5,
        .8,
        1,
        Inf
      ),
      labels = c(
        "0-30",
        "30-50",
        "50-80",
        "80-100",
        "100+"
      ),
      right = FALSE,
      ordered_result = TRUE
    ),
    ami_percentile = ceiling(pct_mfi * 10) / 10,
    tenure = as_factor(OWNERSHP),
    tenure_detail = as_factor(OWNERSHPD),
    housing_cost_pct = case_when(
      OWNERSHP == 2 ~ RENTGRS * 12 / HHINCOME,
      OWNERSHP == 1 ~ OWNCOST * 12 / HHINCOME,
      .default = NA_real_
    ),
    cost_burdened_30 = case_when(
      housing_cost_pct >= .3 ~ TRUE,
      .default = FALSE
    ),
    cost_burdened_50 = case_when(
      housing_cost_pct >= .5 ~ TRUE,
      .default = FALSE
    ),
    rent = RENTGRS,
    rent_groups = cut(rent, breaks = c(0, 500, 1000, 1500, 2000, 2500, 3000, Inf), right = FALSE)
  )


Joining with `by = join_by(YEAR)`


In [6]:
acs_hh <- acs_amis |>  
  filter(PERNUM == 1 & GQ == 1)

acs_srvy_hh <- acs_hh |> 
  as_survey_design(
    weight = HHWT,
    repweights = matches("REPWT[0-9]+$"),
    type = "ACS"
  )

# Generate Summary Statistics
Calculate summary statistics by year, household type, AMI percentile, age group, and tenure status.

In [7]:
# Generate Summary Statistics

# Calculate summary statistics by year, household type, AMI percentile, age group, and tenure status
summary_stats <- acs_hh |>
  filter(HHINCOME != 9999999 & HHINCOME > 0) |>
  # group_by(c(
  #   YEAR,
  #   # household_type_id,
  #   # household_type_description,
  #   # ami_group,
  #   # ami_percentile,
  #   # age_group_id,
  #   # age_group_description,
  #   # tenure,
  #   # tenure_detail,
  #   hhsize
  # )) |> 
  count(YEAR, ami_group, cost_burdened_30, wt = HHWT)

# Display the summary statistics
print(summary_stats)


# A duckplyr data frame: 4
#   variables
    YEAR ami_group cost_burdened_30
   <int> <ord>     <lgl>           
 1  2000 0-30      FALSE           
 2  2000 0-30      TRUE            
 3  2000 30-50     FALSE           
 4  2000 30-50     TRUE            
 5  2000 50-80     FALSE           
 6  2000 50-80     TRUE            
 7  2000 80-100    FALSE           
 8  2000 80-100    TRUE            
 9  2000 100+      FALSE           
10  2000 100+      TRUE            
# ℹ more rows
# ℹ 1 more variable: n <dbl>
# ℹ Use `print(n = ...)` to see more rows


# Calculate Rents by Year

In [8]:
ipumsr::ipums_val_labels(ipums_ddi, "RENT")

# A tibble: 157 × 2
     val lbl                       
   <dbl> <chr>                     
 1    -1 ""                        
 2     0 "N/A"                     
 3     1 "No cash rent (1980-1990)"
 4     2 ""                        
 5     3 ""                        
 6     4 ""                        
 7     5 ""                        
 8     6 ""                        
 9     7 ""                        
10     8 ""                        
# ℹ 147 more rows
# ℹ Use `print(n = ...)` to see more rows

In [9]:
acs_amis |> 
  filter(GQ == 1 & OWNERSHPD %in% c(20, 22)) |> 
  select(RENT) |> 
  distinct() |> 
  arrange(RENT) |> 
  print(n = Inf)

# A duckplyr data frame: 1
#   variable
     RENT
    <int>
  1     4
  2     5
  3    10
  4    20
  5    30
  6    40
  7    50
  8    60
  9    70
 10    80
 11    90
 12   100
 13   110
 14   120
 15   130
 16   140
 17   150
 18   160
 19   170
 20   180
 21   190
 22   200
 23   210
 24   220
 25   230
 26   240
 27   250
 28   260
 29   270
 30   280
 31   290
 32   300
 33   310
 34   320
 35   330
 36   340
 37   350
 38   360
 39   370
 40   380
 41   390
 42   400
 43   410
 44   420
 45   430
 46   440
 47   450
 48   460
 49   470
 50   480
 51   490
 52   500
 53   510
 54   520
 55   530
 56   540
 57   550
 58   560
 59   570
 60   580
 61   590
 62   600
 63   610
 64   620
 65   630
 66   640
 67   650
 68   660
 69   670
 70   680
 71   690
 72   700
 73   710
 74   720
 75   730
 76   740
 77   750
 78   760
 79   770
 80   780
 81   790
 82   800
 83   810
 84   820
 85   830
 86   840
 87   850
 88   860
 89   870
 90   880
 91   890
 92   900
 93   910
 94   920


In [13]:
# get median rent by year

acs_rent_outputs <- acs_srvy_hh |> 
  filter(GQ == 1 & OWNERSHPD %in% c(20, 22)) |> 
  group_by(YEAR) |> 
  summarize(
    hh_rent = survey_quantile(RENTGRS, c(.25, .5, .75)))

# Ownership Costs

First we need to get a list of of ownership types to determine households that have a mortgage.

In [16]:
ipums_val_labels(ipums_ddi, "OWNERSHPD")

# A tibble: 8 × 2
    val lbl                        
  <dbl> <chr>                      
1     0 N/A                        
2    10 Owned or being bought      
3    11 Check mark (owns?)         
4    12 Owned free and clear       
5    13 Owned with mortgage or loan
6    20 Rented                     
7    21 No cash rent               
8    22 With cash rent             

In [22]:
# get median ownership cost by year

acs_own_outputs <- acs_srvy_hh |> 
  filter(GQ == 1 & OWNERSHPD %in% c(10, 11, 13)) |> 
  group_by(YEAR) |> 
  summarize(
    hh_owncost = survey_quantile(OWNCOST, c(0, .25, .5, .75, 1)))

# Export Results
Export the processed data to CSV files in various formats for further analysis.

In [7]:
# Export Results

# Write the summary statistics to a CSV file
write_csv(summary_stats, "data/summary_stats.csv")

# Write the processed AMI data to a CSV file
write_csv(acs_amis, "data/acs_amis.csv")



In [11]:
# Write the combined MFI data to a CSV file ready for eviews
combined_mfi |> 
  arrange(year) |> 
  select(year, mfi_state = estimate) |> 
  write_csv("data/combined_mfi.csv")

In [30]:
# write rent and ownership cost data to csv for eviews by combining and removing SEs

acs_rent_outputs |> 
  left_join(acs_own_outputs, by = join_by(YEAR)) |> 
  select(-ends_with("se")) |> 
  write_csv("data/acs_rent_own.csv")
